In [1]:
import os
import pandas as pd
import multiprocessing
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers, callbacks

2024-07-03 12:33:39.690481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 12:33:39.690598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 12:33:39.825599: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
SPLITS_ID = 4

In [3]:
DATASET_PATH = "/kaggle/input/motocycledataset/Data"

In [4]:
BATCH_SIZE = 64
WIDTH = 224
HEIGHT = 224
SEED = 42

# DATA LOADING

In [5]:
train_csv = os.path.join(DATASET_PATH, f"MotocycleDataset-Splits-{SPLITS_ID}-Train.csv")
test_csv = os.path.join(DATASET_PATH, f"MotocycleDataset-Splits-{SPLITS_ID}-Test.csv")

train_df = pd.read_csv(train_csv, header=None, names=["file_path", "class"])
test_df = pd.read_csv(test_csv, header=None, names=["file_path", "class"])

train_df["file_path"] = train_df["file_path"].apply(lambda x: os.path.join(DATASET_PATH, x))
test_df["file_path"] = test_df["file_path"].apply(lambda x: os.path.join(DATASET_PATH, x))

train_df["class"] = train_df["class"].astype(str)
test_df["class"] = test_df["class"].astype(str)

# PREPROCESSING

In [6]:
image_set = set()

def hash_numpy_array(arr):
    arr_bytes = arr.tobytes()
    hash_obj = hashlib.sha256(arr_bytes)
    hash_hex = hash_obj.hexdigest()
    return hash_hex

def validate_image(image_path, skip_duplicate):
    if not os.path.exists(image_path):
        return False
    if not os.path.isfile(image_path):
        return False
    try:
        with Image.open(image_path) as img:
            img.resize((WIDTH, HEIGHT))
            if skip_duplicate:
                hash_value = hash_numpy_array(np.array(img))
                if hash_value in image_set:
                    return False
                else:
                    image_set.add(hash_value)
        return True

    except Exception as e:
        print(e)
        return False
        
def validate_images_multicore(df, num_processes, skip_duplicate=False):
    with multiprocessing.Pool(num_processes) as pool:
        results = pool.starmap(
            validate_image, 
            zip(df["file_path"], [skip_duplicate] * len(df))
        )
    return df[results]  

In [7]:
train_df = validate_images_multicore(train_df, num_processes=16, skip_duplicate=False)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.277.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.311.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.313.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.323.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/Others/22520968-22520996-22520999-22520929-22521373.Others.567.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/Others/22520968-22520996-22520999-22520929-22521373.Others.568.jpg'


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [8]:
image_set = set()

In [9]:
test_df = validate_images_multicore(test_df, num_processes=16)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


image file is truncated (8 bytes not processed)


# TRAINING

In [10]:
train_data_generator = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
)
test_data_generator = ImageDataGenerator(rescale=1/255)

dataframe_config = {
    'x_col': 'file_path',
    'y_col': 'class',
    'target_size': (HEIGHT, WIDTH),
    'batch_size': BATCH_SIZE,
    'class_mode': 'categorical',
    'shuffle': True,
    'seed': SEED,
    'color_mode': 'rgb',
}

train_generator = train_data_generator.flow_from_dataframe(train_df, **dataframe_config, subset='training')
val_generator = train_data_generator.flow_from_dataframe(train_df, **dataframe_config, subset='validation')
test_generator = train_data_generator.flow_from_dataframe(test_df, **dataframe_config)

Found 22156 validated image filenames belonging to 5 classes.
Found 5539 validated image filenames belonging to 5 classes.
Found 6927 validated image filenames belonging to 5 classes.


In [11]:
base_model = ResNet50(
    include_top=False,  
    weights='imagenet', 
    input_shape=(224, 224, 3) 
)

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))  

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("model.keras", monitor="val_loss", save_best_only=True, verbose=1)
callbacks = [checkpoint]

In [13]:
learning_rate = 0.0001
epochs = 15

In [14]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
I0000 00:00:1720010415.508406     178 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720010415.605641     178 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 16/347 ━━━━━━━━━━━━━━━━━━━━ 16:28 3s/step - accuracy: 0.2935 - loss: 1.5978  

W0000 00:00:1720010460.780051     178 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 776ms/step - accuracy: 0.4849 - loss: 1.2023

W0000 00:00:1720010692.206534     180 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


347/347 ━━━━━━━━━━━━━━━━━━━━ 440s 987ms/step - accuracy: 0.4853 - loss: 1.2016 - val_accuracy: 0.0000e+00 - val_loss: 2.3728
Epoch 2/15


W0000 00:00:1720010757.637193     178 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


347/347 ━━━━━━━━━━━━━━━━━━━━ 295s 837ms/step - accuracy: 0.8259 - loss: 0.4919 - val_accuracy: 0.0388 - val_loss: 3.4625
Epoch 3/15
347/347 ━━━━━━━━━━━━━━━━━━━━ 290s 823ms/step - accuracy: 0.9216 - loss: 0.2382 - val_accuracy: 0.4613 - val_loss: 1.9061
Epoch 4/15
347/347 ━━━━━━━━━━━━━━━━━━━━ 296s 836ms/step - accuracy: 0.9501 - loss: 0.1540 - val_accuracy: 0.4306 - val_loss: 2.9560
Epoch 5/15
347/347 ━━━━━━━━━━━━━━━━━━━━ 301s 852ms/step - accuracy: 0.9668 - loss: 0.1003 - val_accuracy: 0.5136 - val_loss: 2.8240
Epoch 6/15
347/347 ━━━━━━━━━━━━━━━━━━━━ 289s 820ms/step - accuracy: 0.9720 - loss: 0.0853 - val_accuracy: 0.2768 - val_loss: 4.7496
Epoch 7/15
347/347 ━━━━━━━━━━━━━━━━━━━━ 294s 833ms/step - accuracy: 0.9772 - loss: 0.0776 - val_accuracy: 0.5066 - val_loss: 3.4541
Epoch 8/15
347/347 ━━━━━━━━━━━━━━━━━━━━ 290s 818ms/step - accuracy: 0.9804 - loss: 0.0652 - val_accuracy: 0.4880 - val_loss: 3.2151
Epoch 9/15
347/347 ━━━━━━━━━━━━━━━━━━━━ 283s 797ms/step - accuracy: 0.9819 - loss: 0.05

# TESTING

In [21]:
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {accuracy:.6f}")

109/109 ━━━━━━━━━━━━━━━━━━━━ 69s 622ms/step - accuracy: 0.7269 - loss: 1.4558
Test Accuracy: 0.723257


----------------------------------------------------

In [17]:
# test_df = validate_images_multicore(test_df, num_processes=12)

In [18]:
# data_generator = ImageDataGenerator(rescale=1/255)
# dataframe_config = {
#     'dataframe': test_df,
#     'x_col': 'file_path',
#     'y_col': 'class',
#     'target_size': (HEIGHT, WIDTH),
#     'batch_size': BATCH_SIZE,
#     'class_mode': 'categorical',
#     'shuffle': True,
#     'seed': SEED,
#     'color_mode': 'rgb',
# }
# test_generator = data_generator.flow_from_dataframe(**dataframe_config)

In [19]:
# loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
# print(f"Test Accuracy: {accuracy:.2f}")